In [ ]:
#     psutil gives you system stats

#     You can fetch CPU, memory, and disk info

#     You can display it once or loop it over time

#     You can format bytes into readable units

# That’s 90% of the value. The rest is just decoration.


# 먼저 psutil 모듈이 설치되어 있는지 확인
try:
    import psutil
except ImportError:
    print("\n[오류] psutil 모듈을 찾을 수 없습니다.")
    print("이 기능을 사용하려면 'pip install psutil' 명령으로 설치해야 합니다.")
    print("터미널(명령 프롬프트)에서 다음 명령을 실행하세요:")
    print("  pip install psutil")
    exit()

import time
from datetime import datetime
import os  # 파일 정리용

class SystemMonitor:
    def __init__(self):
        self.start_time = time.time()

    def _format_bytes(self, bytes_val):
        """바이트 값을 사람이 읽기 쉬운 단위 (KB, MB, GB)로 변환"""
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if bytes_val < 1024.0:
                return f"{bytes_val:.1f} {unit}"
            bytes_val /= 1024.0
        return f"{bytes_val:.1f} PB"

    def get_cpu_info(self) -> dict:
        """CPU 사용률 및 코어 정보 반환"""
        cpu_percent = psutil.cpu_percent(interval=None)
        cpu_count_logical = psutil.cpu_count()
        cpu_count_physical = psutil.cpu_count(logical=False)

        return {
            'usage_percent': cpu_percent,
            'logical_cores': cpu_count_logical,
            'physical_cores': cpu_count_physical
        }

    def get_memory_info(self) -> dict:
        """RAM 및 스왑 메모리 사용량 정보 반환"""
        memory = psutil.virtual_memory()
        swap = psutil.swap_memory()

        return {
            'ram_total': memory.total,
            'ram_available': memory.available,
            'ram_percent': memory.percent,
            'ram_used': memory.used,
            'swap_total': swap.total,
            'swap_used': swap.used,
            'swap_percent': swap.percent
        }

    def get_disk_info(self, path='/') -> dict:
        """지정된 경로의 디스크 사용량 정보 반환"""
        try:
            disk_usage = psutil.disk_usage(path)
            return {
                'path': path,
                'total': disk_usage.total,
                'used': disk_usage.used,
                'free': disk_usage.free,
                'percent': disk_usage.percent
            }
        except FileNotFoundError:
            return {'path': path, 'error': '경로를 찾을 수 없습니다.'}
        except Exception as e:
            return {'path': path, 'error': str(e)}

    def monitor_system(self, duration_seconds=60, interval_seconds=5):
        """시스템의 CPU, 메모리, 디스크 사용량을 주기적으로 모니터링하고 출력"""
        end_time = time.time() + duration_seconds

        print("\n" + "=" * 60)
        print(f"시스템 모니터링 시작 (총 {duration_seconds}초, {interval_seconds}초 간격)")
        print("=" * 60)

        header_printed = False
        while time.time() < end_time:
            current_time = time.time()

            cpu_info = self.get_cpu_info()
            mem_info = self.get_memory_info()
            disk_info = self.get_disk_info('/')

            timestamp_str = datetime.now().strftime("%H:%M:%S")

            if not header_printed:
                print(f"{'시간':<10} | {'CPU 사용률 (%)':<15} | {'RAM 사용률 (%)':<15} | {'RAM 사용량':<20} | {'디스크 사용률 (%)':<15}")
                print("-" * 90)
                header_printed = True

            cpu_usage_str = f"{cpu_info['usage_percent']:.1f}%"
            ram_percent_str = f"{mem_info['ram_percent']:.1f}%"
            ram_used_str = f"{self._format_bytes(mem_info['ram_used'])} / {self._format_bytes(mem_info['ram_total'])}"
            disk_percent_str = f"{disk_info.get('percent', 'N/A'):.1f}%" if 'error' not in disk_info else f"{disk_info['error']}"

            print(f"{timestamp_str:<10} | {cpu_usage_str:<15} | {ram_percent_str:<15} | {ram_used_str:<20} | {disk_percent_str:<15}")

            time_to_wait = max(0, interval_seconds - (time.time() - current_time))
            time.sleep(time_to_wait)

        print("=" * 60)
        print("시스템 모니터링 종료.")

# --- 사용 예시 ---
if __name__ == "__main__":
    monitor = SystemMonitor()

    # 실시간 모니터링 예시 (주석 해제 시 실행됨)
    # monitor.monitor_system(duration_seconds=30, interval_seconds=5)

    # 현재 시스템 상태 한번만 출력
    print("\n--- 현재 시스템 상태 ---")
    cpu = monitor.get_cpu_info()
    mem = monitor.get_memory_info()
    disk = monitor.get_disk_info('/')

    print(f"CPU: {cpu['usage_percent']:.1f}% 사용 중 ({cpu['logical_cores']} 코어)")
    print(f"RAM: {mem['ram_percent']:.1f}% 사용 중 ({monitor._format_bytes(mem['ram_used'])} / {monitor._format_bytes(mem['ram_total'])})")
    if 'error' not in disk:
        print(f"Disk ('/'): {disk['percent']:.1f}% 사용 중 ({monitor._format_bytes(disk['used'])} / {monitor._format_bytes(disk['total'])})")
    else:
        print(f"Disk ('/'): {disk['error']}")



--- 현재 시스템 상태 ---
CPU: 27.6% 사용 중 (4 코어)
RAM: 62.9% 사용 중 (2.1 GB / 3.7 GB)
Disk ('/'): 4.1% 사용 중 (17.6 GB / 456.9 GB)
